In [119]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)

# Problem 1

In [4]:
data = pd.read_csv('Sdf16_1a.txt', sep='\t')

In [186]:
v = data.groupby('STNAME').agg({'TFEDREV' : 'sum', 'V33' : 'sum'}).reset_index().sort_values('TFEDREV')

v['Revenue Per Student'] = v['TFEDREV'] / v['V33']

v = v.sort_values('Revenue Per Student')

fig = px.bar(v, x='Revenue Per Student', y='STNAME', orientation='h', height = 1000,
             labels=dict(TFEDREV='Revenue', TOTALEXP='Expenditure', STNAME='State'))

fig.layout.title.text = 'Federal Funding By State'
fig.layout.xaxis.title = 'Revenue Per Student'
fig.layout.yaxis.title = ''

fig.show()

Based on the plot above, Washington D.C. and Alaska have the highest federal funding. Utah has the least.

# Problem 2

In [155]:
v = data

fig = px.scatter(data, x='TFEDREV', y='TOTALEXP', hover_name='LEAID', hover_data=['STNAME'], log_x=True, log_y=True,
                labels=dict(TFEDREV='Revenue', TOTALEXP='Expenditure', STNAME='State'))

fig.layout.title.text = 'Federal Funding By District'
fig.layout.xaxis.title = 'Revenue'
fig.layout.yaxis.title = 'Expenditure'

fig.show()

In [176]:
data['debt'] = data['TFEDREV'] - data['TOTALEXP']

In [188]:
v = data.groupby('STNAME').agg({'debt' : 'sum', 'V33' : 'sum'}).reset_index().sort_values('debt', ascending=False)
v['debt'] = v['debt'].apply(abs)

v['Debt Per Student'] = v['debt'] / v['V33']

v = v.sort_values('Debt Per Student')

fig = px.bar(v, x='Debt Per Student', y='STNAME', orientation='h', height = 1000,
             labels=dict(TFEDREV='Revenue', TOTALEXP='Expenditure', STNAME='State'))

fig.layout.title.text = 'Debt By State'
fig.layout.xaxis.title = 'Debt Per Student'
fig.layout.yaxis.title = ''

fig.show()

Washington D.C., New York, and Vermont have the highest debt per student among the states. Idaho has the least.

# Problem 3

In [103]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [167]:
def process_data(val):
    """
    Function to process columns and convert strings to values. The values with equality signs are given the closest
    whole value to the value associeted with the equality sign (i.e. >= 50 -> 50, >50 -> 51). I choose this to handle
    the equalities like this because it creates a less extreme dataset under the uncertainty. '.' and 'PS' values are
    returned as np.nan to impute.
    """
    if isinstance(val, str):
        if val == '.':
            return np.nan
        
        elif val.count('-') > 0: # Averages values if there is a range, indicated by '-'
            return np.mean([float(v) for v in val.split('-')])
        
        elif val[0:2] == 'LE': # less than or equal to
            return float(val.lstrip('LE'))
        elif val[0:2] == 'GE': # greater than or equal to
            return float(val.lstrip('GE'))
        elif val[0:2] == 'LT': # less than
            return float(val.lstrip('LT') - 1)
        elif val[0:2] == 'GT': # greater than
            return float(val.lstrip('GT') + 1)
        
        elif val == 'PS': # Suppressed percent proficient
            return np.nan
        
        return float(val)
        
    elif np.isnan(val):
        return np.nan



In [168]:
mscoers = pd.read_csv('math-achievement-lea-sy2015-16')

# Initialize imputer
imp = IterativeImputer(random_state=0)

In [189]:
v = pd.DataFrame({'Values' : imp.fit_transform(mscores['MBL_MTH00PCTPROF_1516'].apply(process_data).values.reshape(1, -1))[0]})
fig = px.histogram(v, x = 'Values', histnorm='percent',
            labels=dict(TFEDREV='Revenue', TOTALEXP='Expenditure', STNAME='State'))

fig.layout.title.text = 'Distribution of MBL_MTH00PCTPROF_1516 Values'
fig.layout.xaxis.title = 'Score Percent'

fig.show()

# Problem 4

In [137]:
total = sum(data['TFEDREV'])
print(f"Total national education funding is ${total}")

Total national education funding is $55602739138


In [138]:
data['New Funding'] = data['TFEDREV'] * .85

In [143]:
data[['LEAID','STNAME','New Funding']].sort_values('New Funding', ascending=False).head(15).reset_index(drop=True)

,LEAID,STNAME,New Funding
0,3620580,New York,1.478236e+09
1,622710,California,9.276900e+08
2,1709930,Illinois,6.735570e+08
3,1200390,Florida,3.667912e+08
4,1200870,Florida,2.621630e+08
5,4823640,Texas,2.498643e+08
6,4816230,Texas,2.440868e+08
7,1200180,Florida,2.365482e+08
8,3200060,Nevada,2.359838e+08
9,4218990,Pennsylvania,2.306636e+08


# Problem 5

Applying a uniform 15% cut to each school district seemed the most logical and fair way to apply the reduced revenue for the breif analysis of this assignment. I would be concerned applying strategic cuts without a wider literature review of the buget impact on institutions. I definately would not move to cut budgets to more poorly performing schools, as some studies have indicated that poor results are not the fault of the school staff. It also doesn't make since to cut the areas recieving the most budget, as there is no accounting for district size and adjusted cost of the region. The uniform percentage cut preserves the current system, and hopefully policymakers would be working on the back end to create a more research based budget for future funding cycles (I can dream).